In [1]:
# Import required libraries
import pandas as pd
import spacy
import random
from transformers import pipeline
import json
import numpy as np
import re
nlp = spacy.load('en_core_web_sm')

In [2]:
DATA_FOLDER = '../data/'
VARIABLE_FOLDER = '../variables/'
FIGURE_FOLDER = '../reports/'

In [3]:
# Load preprocessed data with numbers and some punctuation marks
df = pd.read_csv(DATA_FOLDER + 'processed/assigned_topics.csv')

texts_list = df.content.to_list()

In [4]:
df.head()

,Unnamed: 0,query,subreddit,type,author,content,date,upvotes,n_comments,len_text,Topic,Group,Keywords,datetime,date_year
0,2,semaglutide,trt,post,Bud1985,Semaglutide. Anyone in here have experience ta...,2023-02-02 14:56:14,4,17,35,5,3,"""semaglutide"", ""appetite"", ""b12"", ""fat"", ""lira...",2023-02-02 14:56:14,2023
1,3,semaglutide,trt,post,Hormonesforme-com,Semaglutide is a revolutionary fat loss medica...,2022-07-30 15:05:59,2,0,523,5,3,"""semaglutide"", ""appetite"", ""b12"", ""fat"", ""lira...",2022-07-30 15:05:59,2022
2,4,semaglutide,trt,post,Hey_its_Jack,Doctors office changed criteria for 'normal le...,2023-05-11 10:38:49,10,66,154,12,3,"""pcos"", ""testosterone"", ""hair"", ""birth"", ""peri...",2023-05-11 10:38:49,2023
3,6,semaglutide,trt,post,Frontyardgoinghard,Advice. 30 year old 6ft 270lbs. I made a post ...,2023-04-28 01:08:07,3,30,207,12,3,"""pcos"", ""testosterone"", ""hair"", ""birth"", ""peri...",2023-04-28 01:08:07,2023
4,7,semaglutide,trt,post,BravoAlphaMikeFoxt,Blood Work and suggestions. 49 yr old 5’7 260#...,2023-05-23 18:07:07,1,4,123,5,3,"""semaglutide"", ""appetite"", ""b12"", ""fat"", ""lira...",2023-05-23 18:07:07,2023


In [5]:
# Import the necessary libraries
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

# Function to generate interactive WordTree webpages
def createWordTrees(query):

    # Initialize the list of sentences
    sentences = []

    # Function to process each sentence and remove stop words and punctuation
    def allSentences(x):
        doc = nlp(str(x))

        # Get the list of English stop words
        stop_words = set(stopwords.words('english'))

        # Get the set of punctuation characters
        punctuation = set(string.punctuation)

        # Iterate through the tokens in the sentence and filter out the stop words and punctuation
        filtered_sentence = [token.text for token in doc if token.text.lower() not in stop_words and token.text not in punctuation]

        # Join the filtered words back to form a sentence
        filtered_sentence = " ".join(filtered_sentence)

        # Append the filtered sentence to the sentences list
        sentences.append(filtered_sentence)

#     sample = pd.read_csv(f"FinalLabelsSample{sampNo}.csv")

#     temp = df.merge(sample, how='inner', left_on='id', right_on='ids')
    temp = df.copy()
    temp['date'] = pd.to_datetime(temp['date'])
    temp.set_index('date', inplace=True)
    print(len(temp))

    #clusterdf = temp[(temp.labels == 1) & (temp.index.month >= 5) & (temp.index.month <=10) & (temp.index.year == 2008)].copy()
#     for i in range(1, 13):
#     print(temp.query == query)
    clusterdf = temp[temp['query'] == query].copy()
    print(len(clusterdf))
    clusterdf['dummy'] = clusterdf['content'].map(
        lambda x: allSentences(x))

    print(len(sentences))
    #sents = clusterdf['title'].to_list()

    # Shuffle the sentences twice thoroughly before using as an input in WordTree
    random.shuffle(sentences)
    random.shuffle(sentences)

    # Limiting the max number of sentences to 10,000 due to performance issues. Higher number of sentences take more memory and time to load and vice-versa
    sentences = sentences[:1000]

    sentsList = [[sent] for sent in sentences]
    use = str(sentsList)[1:-1]
    
    data = calculate_sentiments(sentsList, query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sbaviriseaty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import numpy as np

# Load the sentiment analysis model
sentiment_analysis = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes", return_all_scores=True)

def split_text_into_phrases(text):
    # Split the text into phrases of 512 characters each
    phrase_length = 512
    phrases = [text[i:i + phrase_length] for i in range(0, len(text), phrase_length)]
    return phrases

def find_string_sentiment(text):
    len_text = len(text)

    # Split the text into phrases of 512 characters each
    phrases = split_text_into_phrases(text)

    sents = np.empty((len(phrases), 3))

    for i, phrase in enumerate(phrases):
        # Perform sentiment analysis on the current phrase
        sentiment_i = sentiment_analysis(phrase)
        sents[i, :] = sentiment_i[0][0]['score'], sentiment_i[0][1]['score'], sentiment_i[0][2]['score']

    return np.mean(sents, axis=0)


Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/sbaviriseaty/anaconda3/envs/DHIL_Reddit/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` inst

In [7]:
from transformers import pipeline

def calculate_sentiments(sentsList, query):
    
    phrases = sentsList
    phrases = [phrase[0] for phrase in phrases]
    print(phrases[:5])
    data = [['phrase', 'size', 'value']]

    # Perform sentiment analysis on each phrase and add data to the list
    for phrase in phrases:
        size = 1  # You can customize the size value here if needed
        scores = find_string_sentiment(phrase)
        value = np.argmax(scores)

        data.append([phrase, size, value])
    
    create_page(data, query)

In [8]:
def create_page(data, query):   
#     print(data[0])
#     json_data = json.dumps(data)
#     print(str(data))
#     print(json_data)
    
    # HTML page with Javascript to enable interative web interface
    text = '''
        <html>
          <head>
            <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
            <script type="text/javascript">
              google.charts.load('current', {packages:['wordtree']});
              google.charts.setOnLoadCallback(drawChart);

              function drawChart() {
              console.log('drawing!')
            var data = new google.visualization.arrayToDataTable('''+str(data)+''');

            // Select root word
            var root = document.getElementById('root').value || '';
            var chartType = document.querySelector('input[name = "type"]:checked').value || '';

                var options = {
                  wordtree: {
                    format: 'implicit',
                    type: chartType,
                    word: root,
                    colors: ['red', 'black', 'green']
                  }
                };

            var chart = new google.visualization.WordTree(document.getElementById('wordtree_basic'));
                chart.draw(data, options);
              }

            </script>
          </head>
          <body>
            <p>
            <b><label for="root">Root word:&nbsp;</label></b>
          <input value='''+query+''' id="root" />
          <input type="button" value="go" id="go" onClick="drawChart();" />
            &emsp;&emsp;
            <b><label for="type">Tree type:&nbsp;</label></b>
            <input type="radio" id="r1" name="type" value="double" checked="checked" onClick="drawChart();" /> Double
            <input type="radio" id="r2" name="type" value="suffix" onClick="drawChart();" /> Suffix
            <input type="radio" id="r3" name="type" value="prefix" onClick="drawChart();" /> Prefix

            <br><i>(try: "climate", "energy", "environment" etc.)</i>
            <div id="wordtree_basic" style="width: 1600px; height: 800px;"></div>
            </p>

            <script>
              var getInput = document.getElementById("root");

              getInput.addEventListener("keyup", function(event) {
                if (event.keyCode === 13) {
                  event.preventDefault();
                  document.getElementById("go").click();
                }
              });              
            </script>

          </body>
        </html>

    '''

    file = open(f"Query_{query}.html", "w")
    file.write(text)
    file.close()

#     sentences = []

In [9]:
createWordTrees('wegovy')

12514
1206
1206
['Wegovy Journey Need Know 7 Wegovy injections across 8 weeks   4 injections .25 mg 3 injections .5 mg    lost 20 pounds Wegovy    Sounds amazing right    Well n’t    took .25 mg 4 weeks instantly felt take effect mild diarrhea indigestion would gag thought food lucky eat twice day got frequent migraines ignored symptoms told would take bit time get used Wegovy    48 hours first injection .5 mg pretty bad diarrhea nausea help bland diet zofran fine days    48 hours second injection .5 mg curled ball crying pain abdomen severe diarrhea nausea Zofran could n’t touch worst ever experienced low fever 6 week check prescribing doctor day said likely battling stomach flu n’t caused Wegovy functioning end day    took extra week injections still recovering feeling best eating much third injection 2 weeks second injection    24 hours third injection projectile vomiting dealing severe diarrhea ever terrible indigestion abdomen pain 7/10 fever Three days later getting worse n’t kep